<a href="https://colab.research.google.com/github/gabrielalastra/PYTHON/blob/master/analysis_dadosonline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importando bibliotecas para dados na web
import requests
import pandas as pd


In [ ]:
#funcao que simplifica o processo de importar dados do Cryptowatch
#ref da funcao em https://github.com/cryptowatch/cw-sdk-python 
#va em https://docs.cryptowat.ch/rest-api/#ohlc para o dado publico

def get_historic_price(symbol, exchange='bitfinex', after='2018-09-01'):
    url = 'https://api.cryptowat.ch/markets/{exchange}/{symbol}usd/ohlc'.format(
        symbol=symbol, exchange=exchange)
    resp = requests.get(url, params={
        'periods': '3600',
        'after': str(int(pd.Timestamp(after).timestamp()))
    })
    resp.raise_for_status()
    data = resp.json()
    df = pd.DataFrame(data['result']['3600'], columns=[
        'CloseTime', 'OpenPrice', 'HighPrice', 'LowPrice', 'ClosePrice', 'Volume', 'NA'
    ])
    df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='s')
    df.set_index('CloseTime', inplace=True)
    return df

In [ ]:
#recortar o data das cryptomoedas bitcoin e ether para os ultimos 7 dias

last_week = (pd.Timestamp.now() - pd.offsets.Day(7))
last_week

Timestamp('2021-02-09 13:43:20.370855')

In [ ]:
#definindo a variavel bit como historico de preco da ultima semana
bit = get_historic_price('btc', 'bitstamp', after=last_week)

In [ ]:
#definindo a variavel ether como historico de preco da ultima semana
eth = get_historic_price('eth', 'bitstamp', after=last_week)

In [ ]:
#visualizacao em excel
bit.head()

In [ ]:
bit['ClosePrice'].plot(figsize=(15,7))

In [ ]:
#visualizacao ether
eth.head()

In [ ]:
eth['ClosePrice'].plot(figsize=(15,7))

In [ ]:
#biblioteca para mapas interativos
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [ ]:
output_notebook()

In [ ]:
#para visualizar a variacao do bitcoin com possibilidade de zoom
x1 = figure(x_axis_type='datetime', title='Crypto Prices', width=800)
x1.grid.grid_line_alpha=0.3
x1.xaxis.axis_label = 'Date'
x1.yaxis.axis_label = 'Price'

#a moeda esta definida para a ultima semana
x1.line(bit.index, bit['ClosePrice'], color='#f2a900', legend='Bitcoin')

x1.legend.location = 'top_left'
show(x1)

In [ ]:
#para a cryptomoeda Ether

x1 = figure(x_axis_type='datetime', title='Crypto Prices', width=800)
x1.grid.grid_line_alpha=0.3
x1.xaxis.axis_label = 'Date'
x1.yaxis.axis_label = 'Price'

#a moeda esta definida para a ultima semana
x1.line(eth.index, eth['ClosePrice'], color='#f2a900', legend='Ether')

x1.legend.location = 'top_left'
show(x1)

In [ ]:
#exportando para um excel
writer = pd.ExcelWriter('cryptosfev2021.xlsx')

In [ ]:
#colocando as moedas em colunas separadas
bit.to_excel(writer, sheet_name='Bitcoin')

In [ ]:
eth.to_excel(writer, sheet_name='Ether')

In [ ]:
writer.save()